In [1]:
MODEL = "mistral"

In [5]:
from langchain_community.llms import Ollama
model  = Ollama(model =  MODEL)


In [8]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("data/")
docs= loader.load_and_split()

In [13]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [14]:
chain = prompt | model 

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

' Santiago'

In [15]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\SURYA\anaconda3\envs\RAG\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\SURYA\anaconda3\envs\RAG\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SURYA\.cache\huggingface\hub\models--BAAI--bge-large-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [16]:
embedding = hf.embed_query("hi this is harrison")
len(embedding)

1024

In [18]:
docs[:10]

[Document(metadata={'source': 'data\\anatomy_vol_1.pdf', 'page': 0}, page_content='Anatomy & Physiology \n5FYUCPPL\x01&RVJUZ\x01&EJUJPO\n"O\x010QFO\x01&EVDBUJPO\x013FTPVSDF7PMVNF\x01\x12\x01PG\x01\x14\x1b\x01$IBQUFST\x01\x12\x01\x0e\x01\x12\x12\x01PG\x01\x13\x19\n5IJT\x01NVMUJ\x0eQBSU\x015FYUCPPL\x01&RVJUZ\x01FEJUJPO\x01SFUBJOT\x01UIF\x01PSJHJOBM\nBDBEFNJD\x01DPOUFOU\x01BT\x01QVCMJTIFE\x01CZ\x010QFOTUBY\x01$PMMFHF\r\x01BOE\nVOEFS\x01UIF\x01UFSNT\x01PG\x01UIFJS\x01$SFBUJWF\x01$PNNPOT\x01MJDFOTF\x01\t$$\x0e#:\n\x0f\n-JDFOTF\x1b\x01$$\x0e#:\n*4#/\x1b\x01\x01\x1a\x18\x19\x0e\x12\x0e\x14\x11\x15\x0e\x19\x14\x15\x17\x13\x0e\x12\n\'FBSMFTTMZ\x01DPQZ\r\x01QSJOU\r\x01SFNJYUN'),
 Document(metadata={'source': 'data\\anatomy_vol_1.pdf', 'page': 1}, page_content='OpenStax College  \nRice University 6100 Main Street MS-380 \nHouston, Texas 77005 \nTo learn more about O penStax College, visit http://openstaxcollege.org . \nIndividual print copies and bulk orders can be purchas ed through our website.

In [20]:
import faiss
import numpy as np
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import pickle

# Initialize your HuggingFaceBgeEmbeddings model
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

def extract_text_from_documents(documents):
    return [doc.page_content for doc in documents]  # Access `page_content` to get the text

def generate_embeddings(texts):
    return np.array([hf.embed_query(text) for text in texts])

# 
texts = extract_text_from_documents(docs)

# Generate embeddings
embeddings = generate_embeddings(texts)

# Initialize FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(embeddings)

# Save the index to a file
faiss.write_index(index, "faiss_index.index")

# Save the texts for reference (if needed)
import pickle
with open("texts.pkl", "wb") as f:
    pickle.dump(texts, f)
